In [8]:
# Cell 1: Diagnostic imports
import os
import sys
import importlib

# Get the parent directory path
parent_dir = os.path.dirname(os.getcwd())
print(f"Parent directory: {parent_dir}")

# Force reload modules
if 'fmripy' in sys.modules:
    del sys.modules['fmripy']

try:
    print("\nAttempting direct import:")
    import fmripy.searchlight
    print("Module imported successfully")
    print("Contents:", dir(fmripy.searchlight))
except Exception as e:
    print(f"Direct import error: {str(e)}")

try:
    print("\nAttempting from-import:")
    from fmripy.searchlight import SearchlightAnalysis
    print("Class imported successfully")
except Exception as e:
    print(f"From-import error: {str(e)}")

# Try manual import
print("\nTrying manual import:")
spec = importlib.util.spec_from_file_location(
    "searchlight",
    os.path.join(parent_dir, 'fmripy', 'searchlight.py')
)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
print("Module contents:", dir(module))

Parent directory: /Users/alexe/Kaggle/fmripy

Attempting direct import:
Direct import error: cannot import name 'SearchlightAnalysis' from 'fmripy.searchlight' (/Users/alexe/Kaggle/fmripy/fmripy/searchlight.py)

Attempting from-import:
From-import error: cannot import name 'SearchlightAnalysis' from 'fmripy.searchlight' (/Users/alexe/Kaggle/fmripy/fmripy/searchlight.py)

Trying manual import:
Module contents: ['LogisticRegression', 'Parallel', 'Path', 'SearchLight', 'SearchlightAnalysis', 'StandardScaler', 'StratifiedKFold', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'concat_imgs', 'delayed', 'generate_binary_structure', 'image', 'label', 'load_mni152_brain_mask', 'mean_img', 'multipletests', 'new_img_like', 'np', 'os', 'pd', 'plot_stat_map', 'plt', 'resample_img']


In [1]:
# Cell 1: Imports and Setup
from fmripy import SearchlightAnalysis, ROIConnectivityAnalysis, SeedConnectivityAnalysis
import os
import numpy as np
import matplotlib.pyplot as plt
from nilearn import plotting

# Define common parameters
derivatives_path = "/Volumes/Lito/WADIT/derivatives/fmriprep"
participants_file = '/Volumes/Lito/Whiplash/wadit-source_raw-data/participants.tsv'
output_dir = "analysis_results"

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Define ROIs
rois = [
    '/Users/alexe/WADIT/proproception/regions/EBA_L.nii.gz',
    '/Users/alexe/WADIT/proproception/regions/EBA_R.nii.gz',
    '/Users/alexe/WADIT/proproception/regions/FBA_L.nii.gz',
    '/Users/alexe/WADIT/proproception/regions/FBA_R.nii.gz',
    '/Users/alexe/WADIT/proproception/regions/INS_SPHERE_L.nii.gz',
    '/Users/alexe/WADIT/proproception/regions/INS_SPHERE_R.nii.gz',
]

ImportError: cannot import name 'ROIConnectivityAnalysis' from 'fmripy.roitoroi' (/Users/alexe/Kaggle/fmripy/fmripy/roitoroi.py)

In [ ]:
# Cell 2: Searchlight Analysis (Groups only)
# Initialize searchlight
searchlight = SearchlightAnalysis(
    derivatives_path=derivatives_path,
    participants_file=participants_file,
    output_dir=output_dir,
    n_jobs=4
)

# Run searchlight for session 01 (between groups)
searchlight_results = searchlight.run_analysis(
    session='01',  # Using only session 01 for group comparison
    group1='patient',
    group2='control',
    radius=4.0,
    n_permutations=1000,
    min_cluster_size=10
)

# Print searchlight results summary
print(f"\nSearchlight Analysis Results:")
print(f"Number of clusters found: {searchlight_results['n_clusters']}")

In [ ]:
# Cell 3: ROI-ROI Connectivity Analysis (Groups and Sessions)
# Initialize ROI analysis
roi_analysis = ROIConnectivityAnalysis(
    derivatives_path=derivatives_path,
    participants_file=participants_file,
    output_dir=output_dir
)

# Run ROI analysis for both sessions and groups
roi_results = roi_analysis.run_analysis(
    roi_files=rois,
    sessions=['01', '02'],
    group1='patient',
    group2='control'
)

# Add cluster results from searchlight if desired
cluster_roi_results = roi_analysis.run_analysis(
    roi_files=rois,
    cluster_img_path=f"{output_dir}/searchlight_results_patient_vs_control_ses-01_clusters.nii.gz",
    sessions=['01', '02'],
    group1='patient',
    group2='control'
)

# Print ROI analysis summary
for session in ['01', '02']:
    print(f"\nSession {session} ROI Analysis Results:")
    print(f"Number of significant connections: {np.sum(roi_results[session]['significant'])}")

In [ ]:
# Cell 4: Seed-based Connectivity Analysis (Groups and Sessions)
# Initialize seed analysis
seed_analysis = SeedConnectivityAnalysis(
    derivatives_path=derivatives_path,
    participants_file=participants_file,
    output_dir=output_dir
)

# Run seed analysis for both sessions and groups
seed_results = seed_analysis.run_analysis(
    roi_files=rois,
    sessions=['01', '02'],
    group1='patient',
    group2='control'
)

# Add cluster results from searchlight if desired
cluster_seed_results = seed_analysis.run_analysis(
    roi_files=rois,
    cluster_img_path=f"{output_dir}/searchlight_results_patient_vs_control_ses-01_clusters.nii.gz",
    sessions=['01', '02'],
    group1='patient',
    group2='control'
)

# Print seed analysis summary
for session in ['01', '02']:
    print(f"\nSession {session} Seed Analysis Results:")
    print(f"Analysis completed for {len(rois)} ROIs")